In [1]:
from src.search.paragraphs import DocumentStore, Document
from src.search.passage_ranking import ResponseScorer

2022-06-14 14:15:57.447796: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
host = '<es-host>'

In [3]:
index = 'tmp_es_template' # you have to change the index name to something like '<your-firstname>-blahblah'

In [4]:
storeclient = DocumentStore(host)

In [5]:
storeclient.create_store(index, lang='en', reset_index=True)

INFO:elasticsearch:DELETE http://172.31.18.19:9200/tmp_es_template [status:200 request:0.041s]
INFO:elasticsearch:PUT http://172.31.18.19:9200/tmp_es_template [status:200 request:0.086s]


In [6]:
with open('sample.txt') as f:
    data = [Document(text=s.strip(), id=i) for i, s in enumerate(f.readlines())]

In [7]:
data

[Document(text='machine learning', id=0), Document(text='deep learning', id=1)]

In [8]:
storeclient.add_documents(index=index, documents=data)

INFO:elasticsearch:POST http://172.31.18.19:9200/_bulk?refresh=true [status:200 request:0.011s]


In [9]:
query = 'learn'
#query = 'mediteranen'
#query = 'civilizatio'

In [10]:
print('search results:')
print(storeclient.search(index, query))
storeclient.suggest(index=index, query_string=query)

INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.003s]
INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.003s]


search results:
[SearchResult(paragraph=Document(text='machine learning', id='0'), highlight=['machine <b>learning</b>'], score=0.18232156), SearchResult(paragraph=Document(text='deep learning', id='1'), highlight=['deep <b>learning</b>'], score=0.18232156)]


([{'text': 'learn', 'offset': 0, 'length': 5, 'options': []}],
 [{'text': 'learn', 'offset': 0, 'length': 5, 'options': []}])

In [11]:
scorer = ResponseScorer()

Some weights of the model checkpoint at models/asnq_roberta were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
storeclient.search(index, query, ranker=scorer)

INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.003s]
INFO:root:Score passages...
INFO:root:Prepare data: 0.01s
100%|██████████| 1/1 [00:00<00:00,  3.90it/s]
INFO:root:Inference: 0.26


[SearchResult(paragraph=Document(text='deep learning', id='1'), highlight=['deep <b>learning</b>'], score=0.019679011777043343),
 SearchResult(paragraph=Document(text='machine learning', id='0'), highlight=['machine <b>learning</b>'], score=0.0005291629349812865)]